In [ ]:
import numpy as np
import pandas as pd
import warnings
#warnings.filterwarnings('ignore')

We use data on the Team stats for each regular season game in Euroleague for the seasons 2022-23 and 2023-24. The data for season 2023-24 are up to round 26.\
The data are sourced by Hack A Stat.\
https://hackastat.eu/en/home-page-eng/

**Import data for season 2022-2023**

In [ ]:
df_teams_2223= pd.read_excel(rf'{data}\Teams stats 2022-23.xlsx', sheet_name="TEAMS_GbG", header=2)
df_teams_2223 = df_teams_2223[df_teams_2223["ROUND"]<35]
tags = ["TM TAG", "OPP TAG"]

for t in tags:
    df_teams_2223.loc[df_teams_2223[t] == "EFS", t] = "EFE"
    df_teams_2223.loc[df_teams_2223[t] == "VBC", t] = "VAL"
    df_teams_2223.loc[df_teams_2223[t] == "PAO", t] = "PAN"
    df_teams_2223.loc[df_teams_2223[t] == "FBD", t] = "FEN"
    df_teams_2223.loc[df_teams_2223[t] == "CZV", t] = "CZB"
    df_teams_2223.loc[df_teams_2223[t] == "BSK", t] = "BAS"
    df_teams_2223.loc[df_teams_2223[t] == "MCO", t] = "ASM"

df_teams_2223 = df_teams_2223[df_teams_2223["TM TAG"].isna()==False]
df_teams_2223 = df_teams_2223.iloc[:, :59]

df_teams_2223.loc[df_teams_2223["H/A"] == "H", "H"] = 1
df_teams_2223.loc[df_teams_2223["H/A"] == "A", "H"] = 0
df_teams_2223.loc[df_teams_2223["W/L"] == "W", "W"] = 1
df_teams_2223.loc[df_teams_2223["W/L"] == "L", "W"] = 0
df_teams_2223.drop(['Unnamed: 0', 'RNK', 'TM NAME', 'OPP NAME', 'GM TAG', 'OPP GM TAG', 'DATE', 'GROUP', 'MIN', 'H/A', 'W/L'], axis=1, inplace=True)
df_teams_2223.rename(columns = {'TM TAG':'team', 'OPP TAG':'opp', 'VAL':'PIR', 'TO Ratio': 'TO%'}, inplace = True)
df_teams_2223["AST Ratio"] = df_teams_2223["AST Ratio"]/100
df_teams_2223["TO%"] = df_teams_2223["TO%"]/100
df_teams_2223["SHOOTING CHANCES"] = df_teams_2223["FGA"] + df_teams_2223["TO"] + (0.44*df_teams_2223["FTA"]) # Formula obtained by Hack-a-Stat

rounds = df_teams_2223["ROUND"].max().astype(int)

var = list(df_teams_2223.columns)
for v in var:
    df_teams_2223["OPP " + v] = np.nan
cols = list(df_teams_2223.columns)
df_teams_all = pd.DataFrame(columns=cols)

for r in range(1,rounds+1):
    #print("ROUND", r)
    match = df_teams_2223[df_teams_2223["ROUND"] == r]
    match.reset_index(drop=True, inplace=True)
    for i in range(0,len(match)):
        for j in range(0,len(match)):
            if match["team"][i]==match["opp"][j]:
                for v in var:
                    match["OPP " + v][i] = match[v][j]              
    df_teams_all = pd.concat([df_teams_all, match], ignore_index=True)
    
#df_teams_all.drop(['OPP ROUND', 'OPP team', 'OPP opp', 'OPP H', 'OPP W'], axis=1, inplace=True)
df_teams_all.drop(['OPP ROUND', 'OPP team', 'OPP opp', 'OPP H'], axis=1, inplace=True)
df_teams_2223 = df_teams_all.copy()
del df_teams_all
df_teams_2223["Season"] = 2023
df_teams_2223["+ / -"] = df_teams_2223["PTS"] - df_teams_2223["OPP PTS"]
df_teams_2223["OPP + / -"] = df_teams_2223["OPP PTS"] - df_teams_2223["PTS"]


**Import data for season 2023-2024**

In [ ]:
df_teams_2324 = pd.read_excel(rf'{data}\Teams stats 2023-24.xlsx', sheet_name="TEAMS_GbG", header=2)
df_teams_2324["Season"] = 2024
df_teams_2324.loc[df_teams_2324["H/A"] == "H", "H"] = 1
df_teams_2324.loc[df_teams_2324["H/A"] == "A", "H"] = 0
df_teams_2324.loc[df_teams_2324["W/L"] == "W", "W"] = 1
df_teams_2324.loc[df_teams_2324["W/L"] == "L", "W"] = 0
df_teams_2324.drop(['Unnamed: 0', 'TM NAME', 'OPP NAME', 'DATE', 'GROUP', 'MIN', 'H/A', 'W/L'], axis=1, inplace=True)
df_teams_2324.rename(columns = {'TM TAG':'team', 'OPP TAG':'opp', 'VAL':'PIR', 'OPP VAL':'OPP PIR'}, inplace = True)


print(df_teams_2223.shape)
print(df_teams_2324.shape)

dfs = [df_teams_2223, df_teams_2324]
df_teams = pd.concat(dfs)
df_teams.reset_index(drop=True, inplace=True)
print(df_teams.shape)

# Fill missing data from the 1st season
df_teams.loc[df_teams["FTA/FGA"].isna()==True, "FTA/FGA"] = df_teams["FTA"]/df_teams["FGA"]
df_teams.loc[df_teams["OPP FTA/FGA"].isna()==True, "OPP FTA/FGA"] = df_teams["OPP FTA"]/df_teams["OPP FGA"]
df_teams.loc[df_teams["%2P"].isna()==True, "%2P"] = df_teams["2PTA"]/df_teams["FGA"]
df_teams.loc[df_teams["OPP %2P"].isna()==True, "OPP %2P"] = df_teams["OPP 2PTA"]/df_teams["OPP FGA"]
df_teams.loc[df_teams["%3P"].isna()==True, "%3P"] = df_teams["3PTA"]/df_teams["FGA"]
df_teams.loc[df_teams["OPP %3P"].isna()==True, "OPP %3P"] = df_teams["OPP 3PTA"]/df_teams["OPP FGA"]

df_teams.loc[df_teams["PF 100 Poss"].isna()==True, "PF 100 Poss"] = df_teams["PF"]/df_teams["POSS"]*100
df_teams.loc[df_teams["OPP PF 100 Poss"].isna()==True, "OPP PF 100 Poss"] = df_teams["OPP PF"]/df_teams["OPP POSS"]*100
df_teams.loc[df_teams["DF 100 Poss"].isna()==True, "DF 100 Poss"] = df_teams["DF"]/df_teams["POSS"]*100
df_teams.loc[df_teams["OPP DF 100 Poss"].isna()==True, "OPP DF 100 Poss"] = df_teams["OPP DF"]/df_teams["OPP POSS"]*100

columns_to_drop = df_teams.columns[df_teams.isnull().sum() == len(df_teams_2223)]
df_teams = df_teams.drop(columns=columns_to_drop)

**Append data from both seasons**

In [ ]:
#The remaining missings are OPP stats in the df of the 2nd season. We can retrieve them and we repeat the previous step
var = df_teams.columns[df_teams.isnull().sum() > 0]
var_no_prefix = [column.replace('OPP ', '') for column in var]
var = list(var_no_prefix)

for v in var:
    df_teams_2324["OPP " + v] = np.nan
cols = list(df_teams_2324.columns)
rounds = df_teams_2324["ROUND"].max()
df_teams_all = pd.DataFrame(columns=cols)


for r in range(1,rounds+1):
    #print("ROUND", r)
    match = df_teams_2324[df_teams_2324["ROUND"] == r]
    match.reset_index(drop=True, inplace=True)
    for i in range(0,len(match)):
        for j in range(0,len(match)):
            if match["team"][i]==match["opp"][j]:
                for v in var:
                    match["OPP " + v][i] = match[v][j]              
    df_teams_all = pd.concat([df_teams_all, match], ignore_index=True)
    
df_teams_2324 = df_teams_all.copy()
rounds2324 = df_teams_2324["ROUND"].max()
del df_teams_all 
df_teams_2324 ["Season_Round"] = df_teams_2324["Season"].astype(str) + "_" + df_teams_2324["ROUND"].astype(str)
df_teams_2324["ROUND"] = df_teams_2324["ROUND"] + df_teams_2223["ROUND"].max()
df_teams_2324["ROUND"] = df_teams_2324["ROUND"].astype(int)

df_teams_2223["ROUND"] = df_teams_2223["ROUND"].astype(int)
df_teams_2223["Season_Round"] = df_teams_2223["Season"].astype(str) + "_" + df_teams_2223["ROUND"].astype(str)

dfs = [df_teams_2223, df_teams_2324]
df_teams = pd.concat(dfs)
df_teams.reset_index(drop=True, inplace=True)

df_teams.loc[df_teams["FTA/FGA"].isna()==True, "FTA/FGA"] = df_teams["FTA"]/df_teams["FGA"]
df_teams.loc[df_teams["OPP FTA/FGA"].isna()==True, "OPP FTA/FGA"] = df_teams["OPP FTA"]/df_teams["OPP FGA"]
df_teams.loc[df_teams["%2P"].isna()==True, "%2P"] = df_teams["2PTA"]/df_teams["FGA"]
df_teams.loc[df_teams["OPP %2P"].isna()==True, "OPP %2P"] = df_teams["OPP 2PTA"]/df_teams["OPP FGA"]
df_teams.loc[df_teams["%3P"].isna()==True, "%3P"] = df_teams["3PTA"]/df_teams["FGA"]
df_teams.loc[df_teams["OPP %3P"].isna()==True, "OPP %3P"] = df_teams["OPP 3PTA"]/df_teams["OPP FGA"]

df_teams.loc[df_teams["PF 100 Poss"].isna()==True, "PF 100 Poss"] = df_teams["PF"]/df_teams["POSS"]*100
df_teams.loc[df_teams["OPP PF 100 Poss"].isna()==True, "OPP PF 100 Poss"] = df_teams["OPP PF"]/df_teams["OPP POSS"]*100
df_teams.loc[df_teams["DF 100 Poss"].isna()==True, "DF 100 Poss"] = df_teams["DF"]/df_teams["POSS"]*100
df_teams.loc[df_teams["OPP DF 100 Poss"].isna()==True, "OPP DF 100 Poss"] = df_teams["OPP DF"]/df_teams["OPP POSS"]*100

columns_to_drop = df_teams.columns[df_teams.isnull().sum() == 612]
df_teams = df_teams.drop(columns=columns_to_drop)

cols = list(df_teams.columns)
cols.insert(0, cols.pop(cols.index('Season')))
cols.insert(1, cols.pop(cols.index('Season_Round')))
cols.insert(5, cols.pop(cols.index('W')))
cols.insert(6, cols.pop(cols.index('H')))

df_teams = df_teams.loc[:, cols]

print(df_teams_2223.shape)
print(df_teams_2324.shape)
print(df_teams.shape)

**Import data for future matches and finish the core dataset**

In [ ]:
df_teams_rounds_all = pd.read_excel(rf'{data}\Teams stats 2023-24 ROUNDS ALL.xlsx')
df_teams_rounds_all = df_teams_rounds_all[df_teams_rounds_all["ROUND"] > rounds2324]
df_teams_rounds_all.loc[df_teams_rounds_all["H/A"] == "H", "H"] = 1
df_teams_rounds_all.loc[df_teams_rounds_all["H/A"] == "A", "H"] = 0
df_teams_rounds_all.rename(columns = {'TM TAG':'team', 'OPP TAG':'opp'}, inplace = True)
df_teams_rounds_all.drop(['H/A'], axis=1, inplace=True)
df_teams_rounds_all["Season"] = 2024
df_teams_rounds_all["Season_Round"] = df_teams_rounds_all["Season"].astype(str) + "_" + df_teams_rounds_all["ROUND"].astype(str)
df_teams_rounds_all["ROUND"] = df_teams_rounds_all["ROUND"] + df_teams_2223["ROUND"].max()

cols = list(df_teams_rounds_all.columns)
cols.insert(0, cols.pop(cols.index('Season')))
cols.insert(0, cols.pop(cols.index('Season_Round')))
df_teams_rounds_all = df_teams_rounds_all.loc[:, cols]
dfs = [df_teams, df_teams_rounds_all]
df_teams = pd.concat(dfs)
df_teams.reset_index(inplace=True, drop=True)

df_teams.to_csv(rf'{data}\output\intermediate\df_teams.csv', index=False)